# Simple model

## States

$age_t \in \{57,110\}$: Age.

$d_{t-1} \in \{0,1\}$: Labor market status in beginning of period $t$. 0: retired, 1: working.

$a_{t-1} \in \mathcal{R}_{+}$: Wealth in beginning of period $t$.

## Choices

$C_t \in \mathcal{R}$: Consumption.

$d_t \in \{0,1\}$: Labor market status.

Retirement is absorbing.

## Model

$$V_t(\mathbf{s}_t, \varepsilon_t) = 
\max_{C_t, d_t} \lbrace U(C_t, d_t, \mathbf{s}_t) + \sigma_{\varepsilon} \varepsilon_t(d_t) + 
\beta \mathbf{E}_t \left[\pi_{t+1} V_{t+1}(\mathbf{s}_t, \varepsilon_{t+1}) + (1 - \pi_{t+1})B(a_t) \right]\rbrace$$

$$U(C_t, d_t) = \frac{C_t^{1 - \rho}}{1 - \rho} + \alpha(\mathbf{s}_t) \mathbf{1}_{\{d_t=0\}}$$

$$B(a_t) = \gamma a_t$$

$$\pi_t(age_t)$$

## Parameters

In [1]:
import numpy as np

In [26]:
m = np.linspace(0,10,11)
beta = 4.5

In [31]:
c = m.copy()
for i in reversed(range(len(m))):
    if m[i] > beta:
        c[i] = beta
    else:
        break
a = m - c

In [32]:
c

array([0. , 1. , 2. , 3. , 4. , 4.5, 4.5, 4.5, 4.5, 4.5, 4.5])

In [33]:
m 

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [34]:
a = m - c
a

array([0. , 0. , 0. , 0. , 0. , 0.5, 1.5, 2.5, 3.5, 4.5, 5.5])